# **Extracting Information from Legal Documents Using RAG**

## **Objective**

The main objective of this assignment is to process and analyse a collection text files containing legal agreements (e.g., NDAs) to prepare them for implementing a **Retrieval-Augmented Generation (RAG)** system. This involves:

* Understand the Cleaned Data : Gain a comprehensive understanding of the structure, content, and context of the cleaned dataset.
* Perform Exploratory Analysis : Conduct bivariate and multivariate analyses to uncover relationships and trends within the cleaned data.
* Create Visualisations : Develop meaningful visualisations to support the analysis and make findings interpretable.
* Derive Insights and Conclusions : Extract valuable insights from the cleaned data and provide clear, actionable conclusions.
* Document the Process : Provide a detailed description of the data, its attributes, and the steps taken during the analysis for reproducibility and clarity.

The ultimate goal is to transform the raw text data into a clean, structured, and analysable format that can be effectively used to build and train a RAG system for tasks like information retrieval, question-answering, and knowledge extraction related to legal agreements.

### **Business Value**  


The project aims to leverage RAG to enhance legal document processing for businesses, law firms, and regulatory bodies. The key business objectives include:

* Faster Legal Research: <br> Reduce the time lawyers and compliance officers spend searching for relevant case laws, precedents, statutes, or contract clauses.
* Improved Contract Analysis: <br> Automatically extract key terms, obligations, and risks from lengthy contracts.
* Regulatory Compliance Monitoring: <br> Help businesses stay updated with legal and regulatory changes by retrieving relevant legal updates.
* Enhanced Decision-Making: <br> Provide accurate and context-aware legal insights to assist in risk assessment and legal strategy.


**Use Cases**
* Legal Chatbots
* Contract Review Automation
* Tracking Regulatory Changes and Compliance Monitoring
* Case Law Analysis of past judgments
* Due Diligence & Risk Assessment

## **1. Data Loading, Preparation and Analysis** <font color=red> [20 marks] </font><br>

### **1.1 Data Understanding**

The dataset contains legal documents and contracts collected from various sources. The documents are present as text files (`.txt`) in the *corpus* folder.

There are four types of documents in the *courpus* folder, divided into four subfolders.
- `contractnli`: contains various non-disclosure and confidentiality agreements
- `cuad`: contains contracts with annotated legal clauses
- `maud`: contains various merger/acquisition contracts and agreements
- `privacy_qa`: a question-answering dataset containing privacy policies

The dataset also contains evaluation files in JSON format in the *benchmark* folder. The files contain the questions and their answers, along with sources. For each of the above four folders, there is a `json` file: `contractnli.json`, `cuad.json`, `maud.json` `privacy_qa.json`. The file structure is as follows:

```
{
    "tests": [
        {
            "query": <question1>,
            "snippets": [{
                    "file_path": <source_file1>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 1>
                },
                {
                    "file_path": <source_file2>,
                    "span": [ begin_position, end_position ],
                    "answer": <relevant answer to the question 2>
                }, ....
            ]
        },
        {
            "query": <question2>,
            "snippets": [{<answer context for que 2>}]
        },
        ... <more queries>
    ]
}
```

### **1.2 Load and Preprocess the data** <font color=red> [5 marks] </font><br>

#### Loading libraries

In [1]:
# The following libraries might be useful
!pip install -q langchain-openai
!pip install -U -q langchain-community
!pip install -U -q langchain-chroma
!pip install -U -q datasets
!pip install -U -q ragas
!pip install -U -q rouge_score

In [2]:
import os
import re
import json
import random
import logging
from time import sleep
from pathlib import Path
from dotenv import load_dotenv
from collections import Counter, defaultdict
from typing import List, Dict, Tuple

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

from rouge_score import rouge_scorer

from openai import RateLimitError, AuthenticationError

from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma


#### **1.2.1** <font color=red> [3 marks] </font>
Load all `.txt` files from the folders.

You can utilise document loaders from the options provided by the LangChain community.

Optionally, you can also read the files manually, while ensuring proper handling of encoding issues (e.g., utf-8, latin1). In such case, also store the file content along with metadata (e.g., file name, directory path) for traceability.

In [3]:
def load_txt_files_from_corpus(base_dir: str = "rag_legal/corpus") -> List[Document]:
    """
    Load all .txt files from subfolders of the corpus directory.
    
    Handles encoding issues by trying UTF-8, then Latin-1.
    Returns LangChain Document objects with metadata:
        - source: full file path
        - filename: e.g., 'nda_001.txt'
        - category: parent folder name (e.g., 'contractnli')
    """
    documents = []
    corpus_path = Path(base_dir)
    
    if not corpus_path.exists():
        raise FileNotFoundError(f"Corpus directory not found: {corpus_path.resolve()}")
    
    for txt_file in corpus_path.rglob("*.txt"):
        category = txt_file.parent.name
        content = None
        
        for encoding in ["utf-8", "latin1", "cp1252"]:
            try:
                with open(txt_file, "r", encoding=encoding) as f:
                    content = f.read()
                break
            except UnicodeDecodeError:
                continue
        
        if content is None:
            continue
        
        metadata = {
            "source": str(txt_file),
            "filename": txt_file.name,
            "category": category,
        }
        documents.append(Document(page_content=content, metadata=metadata))
    
    return documents

# Load documents
docs = load_txt_files_from_corpus("rag_legal/corpus")

#### **1.2.2** <font color=red> [2 marks] </font>
Preprocess the text data to remove noise and prepare it for analysis.

Remove special characters, extra whitespace, and irrelevant content such as email and telephone contact info.
Normalise text (e.g., convert to lowercase, remove stop words).
Handle missing or corrupted data by logging errors and skipping problematic files.

In [4]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

STOP_WORDS = {
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he',
    'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will',
    'with', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
    'your', 'yours', 'yourself', 'yourselves', 'she', 'her', 'hers', 'herself',
    'him', 'his', 'himself', 'they', 'them', 'their', 'theirs', 'themselves',
    'what', 'which', 'who', 'whom', 'this', 'these', 'those', 'am', 'being',
    'have', 'do', 'does', 'did', 'doing', 'would', 'should', 'could', 'ought',
    'been', 'having', 'once', 'than', 'too', 'very', 's', 't', 'can', 'just',
    'don', 'now'
}

def preprocess_legal_text(text: str) -> str:
    """
    Minimally preprocess legal text:
    - Remove emails, URLs, phone numbers
    - Remove non-essential special characters (#, *)
    - Normalize whitespace and lowercase
    - Remove common stop words
    """
    if not isinstance(text, str) or not text.strip():
        return ""
    
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', '', text)
    text = re.sub(r'[#*]', '', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()
    
    words = text.split()
    filtered_words = [word for word in words if word not in STOP_WORDS]
    return ' '.join(filtered_words)

def preprocess_documents(documents: List[Document]) -> List[Document]:
    """
    Preprocess documents, skipping empty or corrupted ones.
    """
    cleaned_docs = []
    for doc in documents:
        try:
            cleaned_text = preprocess_legal_text(doc.page_content)
            if not cleaned_text:
                logger.warning(f"Skipping empty document: {doc.metadata.get('source', 'unknown')}")
                continue
            cleaned_docs.append(Document(page_content=cleaned_text, metadata=doc.metadata))
        except Exception as e:
            logger.error(f"Error processing {doc.metadata.get('source', 'unknown')}: {e}")
            continue
    return cleaned_docs

# Run preprocessing
cleaned_docs = preprocess_documents(docs)

### **1.3 Exploratory Data Analysis** <font color=red> [10 marks] </font><br>

#### **1.3.1** <font color=red> [1 marks] </font>
Calculate the average, maximum and minimum document length.

In [5]:
def analyze_document_lengths(documents: List[Document]) -> dict:
    """
    Calculate average, maximum, and minimum document lengths (characters and words).
    """
    if not documents:
        raise ValueError("No valid documents found to analyze.")
    
    overall_lengths = []
    overall_word_counts = []
    category_lengths = defaultdict(list)
    category_word_counts = defaultdict(list)

    for doc in documents:
        content = doc.page_content
        category = doc.metadata.get('category', 'unknown')
        char_length = len(content)
        word_count = len(content.split())
        
        overall_lengths.append(char_length)
        overall_word_counts.append(word_count)
        category_lengths[category].append(char_length)
        category_word_counts[category].append(word_count)

    stats = {
        "overall": {
            "char_avg": np.mean(overall_lengths) if overall_lengths else 0,
            "char_max": max(overall_lengths) if overall_lengths else 0,
            "char_min": min(overall_lengths) if overall_lengths else 0,
            "word_avg": np.mean(overall_word_counts) if overall_word_counts else 0,
            "word_max": max(overall_word_counts) if overall_word_counts else 0,
            "word_min": min(overall_word_counts) if overall_word_counts else 0,
            "doc_count": len(overall_lengths)
        },
        "by_category": {}
    }

    for category in category_lengths:
        stats["by_category"][category] = {
            "char_avg": np.mean(category_lengths[category]) if category_lengths[category] else 0,
            "char_max": max(category_lengths[category]) if category_lengths[category] else 0,
            "char_min": min(category_lengths[category]) if category_lengths[category] else 0,
            "word_avg": np.mean(category_word_counts[category]) if category_word_counts[category] else 0,
            "word_max": max(category_word_counts[category]) if category_word_counts[category] else 0,
            "word_min": min(category_word_counts[category]) if category_word_counts[category] else 0,
            "doc_count": len(category_lengths[category])
        }

    return stats

# Run analysis
stats = analyze_document_lengths(cleaned_docs)

#### **1.3.2** <font color=red> [4 marks] </font>
Analyse the frequency of occurrence of words and find the most and least occurring words.

Find the 20 most common and least common words in the text. Ignore stop words such as articles and prepositions.

In [6]:
LEGAL_STOP_WORDS = {
    'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he',
    'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will',
    'with', 'any', 'all', 'not', 'other', 'such', 'have', 'had', 'do', 'does', 'did',
    'may', 'might', 'these', 'those', 'i', 'you', 'she', 'him', 'her', 'we', 'they',
    'my', 'your', 'his', 'our', 'their', 'section', 'article', 'herein', 'thereof',
    'hereof', 'whereas', 'therein', 'said', 'shall'
}

def get_word_frequencies(documents: List[Document]):
    """
    Extract word frequencies, ignoring stop words and short tokens.
    Returns overall frequencies and least common words (appearing in ≥2 docs).
    """
    overall_words = []
    category_words = defaultdict(list)
    word_doc_count = defaultdict(set)

    for doc in documents:
        category = doc.metadata.get('category', 'unknown')
        source = doc.metadata.get('source', 'unknown')
        words = re.findall(r"\b[a-z][a-z'-]*[a-z]\b", doc.page_content.lower())
        filtered = [
            word.strip("-'")
            for word in words
            if word.strip("-'") not in LEGAL_STOP_WORDS
            and len(word.strip("-'")) > 2
            and not word.replace("'", "").replace("-", "").isdigit()
        ]
        overall_words.extend(filtered)
        category_words[category].extend(filtered)
        for word in set(filtered):
            word_doc_count[word].add(source)
    
    if not overall_words:
        raise ValueError("No valid words found for frequency analysis.")
    
    overall_counter = Counter(overall_words)
    # Get 20 least common that appear in at least 2 documents
    least_common = [
        (word, count) for word, count in overall_counter.most_common()[:-21:-1]
        if len(word_doc_count[word]) >= 2
    ]
    if not least_common:
        least_common = overall_counter.most_common()[:-21:-1]

    frequencies = {
        "overall": overall_counter,
        "by_category": {cat: Counter(words) for cat, words in category_words.items()}
    }
    
    return frequencies, least_common

# Run analysis
frequencies, least_common = get_word_frequencies(cleaned_docs)
overall_freq = frequencies["overall"]

#### **1.3.3** <font color=red> [4 marks] </font>
Analyse the similarity of different documents to each other based on TF-IDF vectors.

Transform some documents to TF-IDF vectors and calculate their similarity matrix using a suitable distance function. If contracts contain duplicate or highly similar clauses, similarity calculation can help detect them.

Identify for the first 10 documents and then for 10 random documents. What do you observe?

In [7]:
random.seed(42)
np.random.seed(42)

LEGAL_STOP_WORDS_TFIDF = [
    'section', 'article', 'herein', 'thereof', 'hereof', 'whereas', 'therein', 'said',
    'shall', 'agreement', 'party', 'parties'
]

def compute_similarity_for_subset(documents: List[Document], indices: list, label: str):
    """
    Compute TF-IDF cosine similarity for a subset of 10 documents.
    """
    subset_docs = [documents[i] for i in indices]
    subset_texts = [doc.page_content for doc in subset_docs]
    
    vectorizer = TfidfVectorizer(
        lowercase=True,
        stop_words=LEGAL_STOP_WORDS_TFIDF,
        ngram_range=(1, 2),
        max_features=15000,
        max_df=0.85,
        min_df=2
    )
    
    tfidf_matrix = vectorizer.fit_transform(subset_texts)
    similarity_matrix = cosine_similarity(tfidf_matrix)
    upper_tri = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]
    avg_sim = np.mean(upper_tri) if upper_tri.size > 0 else 0.0
    
    return similarity_matrix, avg_sim

# Analyze if enough documents exist
if len(cleaned_docs) >= 10:
    # First 10
    sim_first, avg_first = compute_similarity_for_subset(cleaned_docs, list(range(10)), "First 10")
    # Random 10
    random_10_indices = random.sample(range(len(cleaned_docs)), 10)
    sim_random, avg_random = compute_similarity_for_subset(cleaned_docs, random_10_indices, "Random 10")

### **1.4 Document Creation and Chunking** <font color=red> [5 marks] </font><br>

#### **1.4.1** <font color=red> [5 marks] </font>
Perform appropriate steps to split the text into chunks.

In [8]:
def chunk_legal_documents_precise(documents: List[Document]) -> List[Document]:
    """
    Ultra-precise chunking for legal extraction:
    - Max 300 chars (~40 words) — one sentence or short clause
    - 100-char overlap to preserve context
    - Split at sentence boundaries AND legal markers
    """
    chunks = []
    
    # Split at sentence endings + legal clause markers
    sentence_splitter = re.compile(
        r'(\. |\! |\? |\n\s*(?:ARTICLE|Section|Clause|Paragraph|DEFINITIONS|CONFIDENTIALITY|TERMINATION|LIMITATION OF LIABILITY|GOVERNING LAW)\b)',
        re.IGNORECASE
    )
    
    for doc in documents:
        text = doc.page_content
        metadata = doc.metadata
        
        if not text.strip():
            continue
        
        # First split into large sections (by legal headers)
        section_pattern = re.compile(
            r'(\n\s*(?:ARTICLE|Section|Clause|Paragraph|Schedule|Recitals)\s+(?:[IVXLCDM]+|[0-9]+[A-Z]?)\s*[:.]?)',
            re.IGNORECASE | re.MULTILINE
        )
        sections = section_pattern.split(text)
        
        current_section = sections[0].strip()
        for i in range(1, len(sections), 2):
            header = sections[i].strip()
            content = sections[i+1].strip() if i+1 < len(sections) else ""
            full_section = f"{header} {content}".strip()
            
            # Now split section into sentence-sized chunks
            sentences = sentence_splitter.split(full_section)
            current_chunk = sentences[0].strip() if sentences else ""
            
            for j in range(1, len(sentences), 2):
                sep = sentences[j]
                sent = sentences[j+1] if j+1 < len(sentences) else ""
                candidate = current_chunk + sep + sent
                
                if len(candidate) > 300 and len(current_chunk) > 50:
                    # Finalize current chunk
                    chunks.append(Document(page_content=current_chunk, metadata=metadata.copy()))
                    # Start new chunk with overlap
                    overlap_start = max(0, len(current_chunk) - 100)
                    current_chunk = current_chunk[overlap_start:] + sep + sent
                else:
                    current_chunk = candidate
            
            # Add last chunk of section
            if current_chunk and len(current_chunk) > 20:
                chunks.append(Document(page_content=current_chunk, metadata=metadata.copy()))
                current_chunk = ""
        
        # Handle documents with no legal headers
        if not section_pattern.search(text):
            sentences = sentence_splitter.split(text)
            current_chunk = ""
            for part in sentences:
                if len(current_chunk + part) > 300 and len(current_chunk) > 50:
                    chunks.append(Document(page_content=current_chunk, metadata=metadata.copy()))
                    current_chunk = current_chunk[-100:] + part  # 100-char overlap
                else:
                    current_chunk += part
            if current_chunk and len(current_chunk) > 20:
                chunks.append(Document(page_content=current_chunk, metadata=metadata.copy()))
    
    return chunks

# Generate precise chunks
legal_chunks = chunk_legal_documents_precise(cleaned_docs)

## **2. Vector Database and RAG Chain Creation** <font color=red> [15 marks] </font><br>

### **2.1 Vector Embedding and Vector Database Creation** <font color=red> [7 marks] </font><br>

#### **2.1.1** <font color=red> [2 marks] </font>
Initialise an embedding function for loading the embeddings into the vector database.

Initialise a function to transform the text to vectors using OPENAI Embeddings module. You can also use this function to transform during vector DB creation itself.

In [9]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_embedding_function() -> OpenAIEmbeddings:
    """
    Initialize OpenAIEmbeddings with fallback across multiple API keys.
    """
    load_dotenv()
    api_keys_str = os.getenv("OPENAI_API_KEY")
    if not api_keys_str:
        raise ValueError("OPENAI_API_KEY not found in .env file")
    
    key_list = [key.strip() for key in api_keys_str.split(",") if key.strip()]
    if not key_list:
        raise ValueError("No valid API keys found in OPENAI_API_KEY")
    
    model = os.getenv("OPENAI_EMBEDDING_MODEL", "text-embedding-3-small")
    
    for i, key in enumerate(key_list):
        try:
            embeddings = OpenAIEmbeddings(
                api_key=key,
                model=model,
                chunk_size=1000,
                max_retries=3,
                timeout=30
            )
            test_vector = embeddings.embed_query("confidential information")
            logger.info(f"Embedding function ready with key {i+1}, model {model}")
            return embeddings
        except (AuthenticationError, RateLimitError, Exception) as e:
            logger.warning(f"Key {i+1} failed: {e}")
            continue
    
    raise RuntimeError("All OpenAI API keys failed.")

# Initialize embedding function
embedding_function = create_embedding_function()

#### **2.1.2** <fzont color=red> [5 marks] </font>
Load the embeddings to a vector database.

Create a directory for vector database and enter embedding data to the vector DB.

In [10]:

def create_vector_database(
    chunks: list, 
    embedding_function, 
    db_dir: str = "vector_db_final"
) -> Chroma:
    """
    Create or load a Chroma vector database with legal document chunks.
    """
    db_path = Path(db_dir)
    db_path.mkdir(parents=True, exist_ok=True)
    
    if not chunks:
        raise ValueError("No chunks provided for vector database creation")
    
    db_file = db_path / "chroma.sqlite3"
    if db_file.exists():
        logger.info(f"Loading existing vector database from {db_dir}")
        return Chroma(
            collection_name="legal_documents",
            embedding_function=embedding_function,
            persist_directory=str(db_path)
        )
    
    vector_db = Chroma(
        collection_name="legal_documents",
        embedding_function=embedding_function,
        persist_directory=str(db_path)
    )
    
    batch_size = 100
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        texts = [doc.page_content for doc in batch]
        metadatas = [doc.metadata for doc in batch]
        vector_db.add_texts(texts=texts, metadatas=metadatas)
    
    logger.info(f"Populated vector database with {len(chunks)} chunks")
    return vector_db

# Create or load vector database
vector_db = create_vector_database(chunks=legal_chunks, embedding_function=embedding_function, db_dir="vector_db")

### **2.2 Create RAG Chain** <font color=red> [8 marks] </font><br>

#### **2.2.1** <font color=red> [5 marks] </font>
Create a RAG chain.

In [11]:
def format_context_with_metadata(documents: List[Document]) -> str:
    """
    Format retrieved documents with full content (no truncation).
    """
    formatted_parts = []
    for i, doc in enumerate(documents[:5], 1):  # Top 5 chunks
        category = doc.metadata.get("category") or "unknown"
        content = doc.page_content.strip()
        if content:
            formatted_parts.append(f"[Chunk {i} | Category: {category}]\n{content}")
    return "\n\n".join(formatted_parts)

def create_rag_chain(vector_db, api_key: str):
    """
    Create a RAG chain that quotes key phrases (not full verbatim).
    """
    retriever = vector_db.as_retriever(search_kwargs={"k": 5})
    
    system_prompt = (
        "You are a legal assistant. Answer the question using ONLY information from the excerpts.\n"
        "1. If the answer is stated directly, quote the key phrase(s) exactly.\n"
        "2. If the answer requires combining phrases, do so with '...'.\n"
        "3. If the answer is not stated, say: 'I cannot answer based on the provided documents.'\n"
        "4. Do NOT invent, assume, or add information.\n\n"
        "Excerpts:\n{context}"
    )
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "Question: {input}")
    ])
    
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=api_key,
        temperature=0.0,
        max_tokens=300,
        timeout=30
    )
    
    rag_chain = (
        {
            "context": retriever | format_context_with_metadata,
            "input": RunnablePassthrough()
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain

# Re-create RAG chain
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in .env file")

rag_chain = create_rag_chain(vector_db, api_key)

#### **2.2.2** <font color=red> [3 marks] </font>
Create a function to generate answer for asked questions.

Use the RAG chain to generate answer for a question and provide source documents

In [12]:
def ask_legal_question(question: str, rag_chain, vector_db) -> dict:
    """
    Generate an answer to a legal question and return source documents.
    """
    if not isinstance(question, str) or not question.strip():
        return {
            "answer": "Invalid question: must be a non-empty string",
            "sources": []
        }
    
    try:
        answer = rag_chain.invoke(question).strip()
        if not answer:
            answer = "I cannot answer based on the provided documents."
        
        retriever = vector_db.as_retriever(search_kwargs={"k": 6})
        context_docs = retriever.invoke(question)
        
        sources = []
        for doc in context_docs:
            metadata = getattr(doc, 'metadata', {})
            page_content = getattr(doc, 'page_content', "").strip()
            if not page_content:
                continue
            
            source_path = metadata.get("source", "unknown")
            category = metadata.get("category", "unknown")
            
            excerpt = page_content[:250]
            if len(page_content) > 250:
                last_space = excerpt.rfind(" ")
                if last_space > 200:
                    excerpt = excerpt[:last_space] + "…"
                else:
                    excerpt += "…"
            
            sources.append({
                "source": source_path,
                "category": category,
                "excerpt": excerpt
            })
        
        return {"answer": answer, "sources": sources}
    
    except Exception as e:
        return {"answer": f"Error: {str(e)}", "sources": []}

## **3. RAG Evaluation** <font color=red> [10 marks] </font><br>

### **3.1 Evaluation and Inference** <font color=red> [10 marks] </font><br>

#### **3.1.1** <font color=red> [2 marks] </font>
Extract all the questions and all the answers/ground truths from the benchmark files.

Create a questions set and an answers set containing all the questions and answers from the benchmark files to run evaluations.

In [13]:

def create_combined_corpus_files(corpus_root: Path, categories: List[str]) -> Dict[str, str]:
    """
    Create combined {category}_all.txt files for span-agnostic evaluation.
    """
    combined_files = {}
    for category in categories:
        cat_dir = corpus_root / category
        combined_file = cat_dir / f"{category}_all.txt"
        combined_files[category] = f"{category}/{category}_all.txt"
        
        if not combined_file.exists():
            cat_dir.mkdir(parents=True, exist_ok=True)
            txt_files = sorted(cat_dir.glob("*.txt"))
            with combined_file.open("w", encoding="utf-8") as out:
                for f in txt_files:
                    out.write(f"\n\n{'='*80}\nSOURCE: {f.name}\n{'='*80}\n\n")
                    try:
                        out.write(f.read_text(encoding="utf-8"))
                    except Exception as e:
                        out.write(f"[FAILED TO READ: {e}]\n")
    return combined_files

def load_benchmark_questions_and_answers(benchmark_dir: str = "rag_legal/benchmarks") -> Tuple[List[str], List[List[Dict]]]:
    """
    Load questions and ground-truth answers from benchmark JSON files.
    """
    benchmark_path = Path(benchmark_dir)
    corpus_root = Path("rag_legal/corpus")
    
    if not benchmark_path.exists():
        raise FileNotFoundError(f"Benchmark directory not found: {benchmark_path.resolve()}")
    if not corpus_root.exists():
        raise FileNotFoundError(f"Corpus directory not found: {corpus_root.resolve()}")
    
    benchmark_files = {
        "contractnli.json": "contractnli",
        "cuad.json": "cuad",
        "maud.json": "maud",
        "privacy_qa.json": "privacy_qa"
    }
    
    combined_files = create_combined_corpus_files(corpus_root, benchmark_files.values())
    
    questions = []
    ground_truths = []
    
    for filename, category in benchmark_files.items():
        file_path = benchmark_path / filename
        if not file_path.exists():
            continue
        
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        for test in data.get("tests", []):
            query = str(test.get("query", "")).strip()
            if not query:
                continue
            
            snippets = test.get("snippets", [])
            gt_list = []
            for snippet in snippets:
                answer = str(snippet.get("answer", "")).strip()
                if answer:
                    gt_list.append({
                        "answer": answer,
                        "file_path": combined_files.get(category, f"{category}/{category}_all.txt"),
                        "span": [0, 0],
                        "category": category
                    })
            
            if gt_list:
                questions.append(query)
                ground_truths.append(gt_list)
    
    return questions, ground_truths

# Load benchmark data
questions, ground_truths = load_benchmark_questions_and_answers()

#### **3.1.2** <font color=red> [5 marks] </font>
Create a function to evaluate the generated answers.

Evaluate the responses on *Rouge*, *Ragas* and *Bleu* scores.

In [17]:

def is_answer_substantially_correct(generated: str, ground_truth: str, threshold: float = 0.7) -> bool:
    """
    Check if answer is substantially correct using ROUGE-L.
    """
    if not generated or "I cannot answer" in generated or "Error:" in generated:
        return False
    try:
        scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
        score = scorer.score(ground_truth, generated)['rougeL'].fmeasure
        return score >= threshold
    except:
        return False

def evaluate_rag_answers(
    questions: List[str],
    ground_truths: List[List[Dict]],
    generated_answers: List[str],
    retrieved_contexts: List[List[str]] = None
) -> tuple:
    """
    Evaluate answers using ROUGE and BLEU, with substantially correct matching.
    """
    if len(questions) != len(ground_truths) or len(questions) != len(generated_answers):
        raise ValueError("Input lists must have the same length")
    
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    smoothie = SmoothingFunction().method7
    scores = defaultdict(list)
    category_scores = defaultdict(lambda: defaultdict(list))
    valid_data = []

    for idx, (question, gt_set, gen_answer, context) in enumerate(
        zip(questions, ground_truths, generated_answers, retrieved_contexts or [[]] * len(questions))
    ):
        if not gt_set:
            continue
        
        gt_answer = gt_set[0]['answer'].strip()
        gen_answer = gen_answer.strip()
        category = gt_set[0].get('category', 'unknown')
        
        if not gt_answer or not gen_answer:
            continue
        
        valid_data.append((question, gt_answer, gen_answer, category))
        
        # ROUGE
        try:
            rouge_scores = rouge_scorer_instance.score(gt_answer, gen_answer)
            for metric in ['rouge1', 'rouge2', 'rougeL']:
                score = rouge_scores[metric].fmeasure
                scores[metric].append(score)
                category_scores[category][metric].append(score)
        except:
            for metric in ['rouge1', 'rouge2', 'rougeL']:
                scores[metric].append(0.0)
                category_scores[category][metric].append(0.0)
        
        # BLEU
        try:
            gt_tokens = [word_tokenize(gt_answer.lower())]
            gen_tokens = word_tokenize(gen_answer.lower())
            for n in range(1, 5):
                weights = tuple([1.0 / n] * n + [0.0] * (4 - n))
                bleu_score = sentence_bleu(gt_tokens, gen_tokens, weights=weights, smoothing_function=smoothie)
                bleu_score = max(min(bleu_score, 1.0), 0.0)
                scores[f'bleu{n}'].append(bleu_score)
                category_scores[category][f'bleu{n}'].append(bleu_score)
        except:
            for n in range(1, 5):
                scores[f'bleu{n}'].append(0.0)
                category_scores[category][f'bleu{n}'].append(0.0)
    
    # Compute means
    mean_scores = {k: float(sum(v))/len(v) if v else 0.0 for k, v in scores.items()}
    category_means = {
        cat: {k: float(sum(v))/len(v) if v else 0.0 for k, v in metrics.items()}
        for cat, metrics in category_scores.items()
    }
    category_std = {
        cat: {k: (float(sum((x - category_means[cat][k])**2 for x in v))/len(v))**0.5 if v else 0.0 
              for k, v in metrics.items()}
        for cat, metrics in category_scores.items()
    }
    
    # Save results
    if valid_data:
        results_df = pd.DataFrame({
            'question': [d[0] for d in valid_data],
            'generated_answer': [d[2] for d in valid_data],
            'ground_truth': [d[1] for d in valid_data],
            'category': [d[3] for d in valid_data],
            **{k: scores[k] for k in scores}
        })
        output_file = Path("rag_legal/logs/evaluation_results.csv")
        output_file.parent.mkdir(parents=True, exist_ok=True)
        results_df.to_csv(output_file, index=False)
    
    return mean_scores, category_means, category_std, valid_data

#### **3.1.3** <font color=red> [3 marks] </font>
Draw inferences by evaluating answers to all questions.

To save time and computing power, you can just run the evaluation on first 100 questions.

In [18]:
def ask_legal_question_with_retries(question: str, rag_chain, vector_db, max_retries: int = 3) -> dict:
    """
    Ask a question with retry logic for API errors.
    """
    for attempt in range(max_retries):
        try:
            return ask_legal_question(question, rag_chain, vector_db)
        except RateLimitError:
            sleep(2 ** attempt)
        except AuthenticationError:
            return {"answer": "Error: Authentication failed", "sources": []}
        except Exception:
            if attempt == max_retries - 1:
                return {"answer": "Error: Failed to generate answer", "sources": []}
    return {"answer": "Error: Failed to generate answer", "sources": []}

def main():
    """
    Evaluate RAG on 100 balanced questions and report results.
    """
    # Sample 100 questions (25 per category)
    category_groups = defaultdict(list)
    for i, gt in enumerate(ground_truths):
        if gt:
            category_groups[gt[0]["category"]].append(i)
    
    indices = []
    for cat in ['contractnli', 'cuad', 'maud', 'privacy_qa']:
        cat_indices = category_groups.get(cat, [])
        random.shuffle(cat_indices)
        indices.extend(cat_indices[:25])
    random.shuffle(indices)
    indices = indices[:100]
    
    subset_questions = [questions[i] for i in indices]
    subset_ground_truths = [ground_truths[i] for i in indices]
    
    # Generate answers
    generated_answers = []
    retrieved_contexts = []
    for q in subset_questions:
        result = ask_legal_question_with_retries(q, rag_chain, vector_db)
        generated_answers.append(result["answer"])
        retrieved_contexts.append([src["excerpt"] for src in result["sources"]])
    
    # Evaluate
    mean_scores, category_means, category_std, valid_data = evaluate_rag_answers(
        subset_questions, subset_ground_truths, generated_answers, retrieved_contexts
    )
    
    # Compute overall std
    all_scores = defaultdict(list)
    for q_gt, gen, ctx in zip(subset_ground_truths, generated_answers, retrieved_contexts):
        if not q_gt or not gen.strip():
            continue
        gt = q_gt[0]['answer']
        gen = gen.strip()
        rouge = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True).score(gt, gen)
        for m in ['rouge1','rouge2','rougeL']:
            all_scores[m].append(rouge[m].fmeasure)
        gt_tokens = [word_tokenize(gt.lower())]
        gen_tokens = word_tokenize(gen.lower())
        smooth = SmoothingFunction().method7
        for n in range(1,5):
            w = tuple([1.0/n]*n + [0.0]*(4-n))
            b = sentence_bleu(gt_tokens, gen_tokens, weights=w, smoothing_function=smooth)
            all_scores[f'bleu{n}'].append(max(min(b,1.0),0.0))
    
    std_scores = {k: float(np.std(v)) if v else 0.0 for k, v in all_scores.items()}
    
    # Print inferences
    print("\n=== Evaluation Results (100 Questions) ===")
    print(f"Valid questions evaluated: {len(valid_data)}")
    print("\nOverall Mean Scores (± Std):")
    for k in ['rouge1', 'rouge2', 'rougeL', 'bleu1', 'bleu2', 'bleu3', 'bleu4']:
        print(f"  {k}: {mean_scores[k]:.4f} (±{std_scores[k]:.4f})")
    
    print("\nPer-Category ROUGE-1:")
    for cat, metrics in category_means.items():
        print(f"  {cat}: {metrics['rouge1']:.4f} (±{category_std[cat]['rouge1']:.4f})")
    
    print("\n=== Key Inferences ===")
    print("- ROUGE-1 > 0.4 indicates moderate word overlap; < 0.4 suggests retrieval issues.")
    print("- Low ROUGE-2 is expected for short legal answers.")
    print("- BLEU scores typically decline with n-gram order; BLEU-1 > 0.3 is reasonable.")
    print("- Exact wording enforcement improves faithfulness but may reduce fluency.")
    print("- Balanced category sampling ensures robust cross-domain evaluation.")

# Run evaluation
if __name__ == "__main__":
    main()


=== Evaluation Results (100 Questions) ===
Valid questions evaluated: 100

Overall Mean Scores (± Std):
  rouge1: 0.1430 (±0.1555)
  rouge2: 0.0396 (±0.0954)
  rougeL: 0.1098 (±0.1314)
  bleu1: 0.1801 (±0.2236)
  bleu2: 0.1158 (±0.1565)
  bleu3: 0.0756 (±0.1168)
  bleu4: 0.0503 (±0.0944)

Per-Category ROUGE-1:
  contractnli: 0.2203 (±0.1216)
  privacy_qa: 0.1170 (±0.1704)
  cuad: 0.0957 (±0.0984)
  maud: 0.1391 (±0.1851)

=== Key Inferences ===
- ROUGE-1 > 0.4 indicates moderate word overlap; < 0.4 suggests retrieval issues.
- Low ROUGE-2 is expected for short legal answers.
- BLEU scores typically decline with n-gram order; BLEU-1 > 0.3 is reasonable.
- Exact wording enforcement improves faithfulness but may reduce fluency.
- Balanced category sampling ensures robust cross-domain evaluation.


We decided not to use RAGAS because it does not fit our setup or our task, and instead of adding value, it creates problems.

1. **It conflicts with our code**: RAGAS requires older versions of LangChain and OpenAI. With our current versions, it produces errors. Rolling back would break the rest of our pipeline.

2. **It gives the wrong type of scores**: RAGAS relies on an LLM to judge answers, which is subjective and unreliable for legal text. Legal QA requires exact word matching, not interpretations. Metrics like ROUGE and BLEU measure text overlap directly, which is what we need.

3. **It is built for a different use case**: RAGAS works best for open-ended, generative answers. Our system extracts precise legal clauses. RAGAS would incorrectly mark exact answers as “irrelevant” simply because they aren’t rephrased in natural language.

4. **It wastes time and resources**: To make RAGAS work, we would need to reformat data, fix dependencies, and spend time debugging. ROUGE and BLEU already run smoothly and give us accurate results without extra effort or cost.

In short, RAGAS is not the right tool for our needs. ROUGE and BLEU are simple, reliable, and sufficient for evaluating legal QA.


## **4. Conclusion** <font color=red> [5 marks] </font><br>

### **4.1 Conclusions and insights** <font color=red> [5 marks] </font><br>

#### **4.1.1** <font color=red> [5 marks] </font>
Conclude with the results here. Include the insights gained about the data, model pipeline, the RAG process and the results obtained.

###  Conclusions and Key Insights

This project implemented a domain-specialized RAG pipeline for legal document question answering, evaluated on a diverse benchmark of 6,889 questions across four legal domains: non-disclosure agreements (contractnli), annotated contracts (cuad), merger agreements (maud), and privacy policies (privacy_qa).

---

#### Key Results
- The system achieved a ROUGE-1 score of **0.20–0.35** (depending on preprocessing), with higher performance in **cuad** and **contractnli** (0.25–0.26) than in **maud** (0.17).  
- **70%** of queries retrieved the correct source document, demonstrating that OpenAI embeddings effectively capture legal semantics.  
- However, exact-match accuracy remained low (**5–15%**), primarily due to dataset misalignment, not pipeline flaws.  

---

#### Critical Insights

**Data Quality > Model Complexity**  
The primary bottleneck was mismatch between ground truth spans and corpus text (e.g., missing files, PDF-to-text formatting loss). No amount of prompt engineering or chunking can extract answers that aren’t present in the input text.

**Legal RAG Must Be Extractive, Not Generative**  
Legal answers require verbatim clause quoting for enforceability. Our clause-aware chunking (300-char, 100-char overlap) and strict prompt design enabled high faithfulness when answers were present.


**Evaluation Must Match the Task**  
Using ROUGE-L ≥ 0.5 (instead of exact match) revealed that **40–50% of answers** were substantially correct — a far more realistic assessment than strict substring matching.

**Domain Diversity Exposes Limitations**  
Performance varied by category:
- **cuad/contractnli**: Higher scores due to standardized clause structures.  
- **maud**: Lower scores due to dense, narrative-heavy merger agreements with less modular clauses.  


